VERİ SETİNİ TANIMLAYALIM: 1896 Atina Olimpiyatları'ndan 2016 Rio Olimpiyatları'na kadar uzanan ve oyunların kayıtlarına sahip olan Olimpiyat Oyunlarının kapsamlı bir veri kümesidir.Her örnek, bireysel bir Olimpik etkinlikte (sporcu etkinlikleri) yarışan bireysel bir sporcuya karşılık gelir.

İlk olarak kütüphaneleri yükleyelim:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Olimpiyat Sporcuları Veri setini yükleyelim:

In [ ]:
df1 = pd.read_csv("/df1.csv", index_col=1).drop(columns=["Unnamed: 0"])
df1

NOC (National Olympic Committee) Bölge Veri setini yükleyelim:

In [ ]:
df2 = pd.read_csv("/noc_regions.csv")
df2

Her iki veri setini de detaylı inceleyip istatistiksel değerlerini inceleyelim.


In [ ]:
df1.info()

In [ ]:
df1.describe()

In [ ]:
df2.info()

In [ ]:
df2.describe()

Her iki veri setinde de NOC sütunu olduğundan veri setlerini bu sütuna göre birleştirelim:

In [ ]:
df= pd.merge(df1, df2, on='NOC')
df

Bu projemizin odak noktası yalnızca yaz olimpiyatları olacağı için, veri kümemizden tüm kış olimpiyatlarını filtreleyelim ve veri kümemiz üzerinde bazı temel analizler yapalım.

In [ ]:
df = df[df["Season"]=="Summer"]
df

filtreleme işleminden sonra hangi sütunlarda kaç tane NULL (NaN) değer var bakalım:

In [ ]:
df.isnull().sum()

3 tip madalya olduğundan kazanılan madalyaların yüzdelik oranlarına bakalım:

In [ ]:
df.Medal.value_counts(normalize = True)

Kayıp gözlemler tüm verinin yüzde kaçını oluşturuyor bakalım:

In [ ]:
missing_percentage= 100*(df.isna().sum().sort_values(ascending = False)/len(df))
missing_percentage[missing_percentage!=0]

Notes sütununu veri setinden kaldıralım:

In [ ]:
df.drop(columns=["notes"], inplace=True)

weight sütunun veri tipini detayl inceleyelim: 53764 tane kayıp veri olduğunu göreceğiz.

In [ ]:
df['Weight'].value_counts()

weight sütunundaki ? görülen yerler artık NaN olsun istiyoruz.

In [ ]:
df['Weight'] = df['Weight'].replace(['?'], np.nan)

eksik değerleri tekrar kontrol edelim:ilk kontrolde weight sütununda hiç eksik değer yok gibi gözüküyordu fakat şimdi düzeltildi.

In [ ]:
df.isnull().sum()

Age sütunundaki tüm boş değerleri ortalama ile dolduralım:

In [ ]:
df['Age'].fillna(df['Age'].mean(),inplace=True)

Height sütunundaki tüm boş değerleri ortalama ile dolduralım:

In [ ]:
df['Height'].fillna(df['Height'].mean(),inplace=True)

weight sütunun veri tipini string den floata çevirellim:

In [ ]:
df['Weight'] = df['Weight'].astype(float)

şimdi de weight sütunundaki eksik değerleri ortalama ile dolduralım:

In [ ]:
df['Weight'].fillna(df['Weight'].mean(), inplace=True)

Region sütunundaki farklı değerlere bakalım:

In [ ]:
df["region"].unique()

222203 tane veri den sadece 21 tane region verisi eksikti o yüzden bu regıon sütunundaki 21 tane eksik değeri komple veri setinden kaldıralım.

In [ ]:
df.region.dropna(inplace=True)

3 farklı madalya tipi olduğundan ve veri setinde oldukça fazla sayıda medal sütununda NaN değer olduğundan bu NaN değerleri 'madalya kazanamadı' olarak dolduralım:

In [ ]:
df['Medal'].fillna("Medal Not Won", inplace=True)

veri setlerini birleştirip kısmen temizlik işlemleri yapıldıktan sonra birbirleri ile aynı olan satırlar varmı bakalım:

In [ ]:
df[df.duplicated()]

toplam kaç adet duplicate satırlar var?:

In [ ]:
df.duplicated().sum()

tekrar eden değerleri(satırları) veri setinden kaldıralım:

In [ ]:
df.drop_duplicates(inplace=True)

veri setinde tekrar eden değerler kaldı mı kontrol edelim:

In [ ]:
df.duplicated().sum()

Yaz olimpiyatlarına en çok sporcu gönderen ülke hangisidir ve her bir ülke toplam bu yıllar arasında kaç sporcu yollamıştır?:

In [ ]:
athlete_count = df.Team.value_counts()
athlete_count

yaz olimpiyatlarına en çok sporcu gönderen ilk 10 ülkeyi bar chart ile görselleştirelim:

In [ ]:
plt.figure(figsize=[18,8])
sns.barplot(x=athlete_count[:10].index, y=athlete_count[:10])
plt.title("Countries Send the Most Athletes to the Olympics")
plt.xlabel("Countries")
plt.ylabel("Athlete Count");

veri setinden 'madalya kazanamadı' olan kişileri çıkarıp sadece altın, gümüş ve bronz madalya kazananları görelim (yani madalya kazananların listesini görelim):

In [ ]:
df[df.Medal != 'Medal Not Won']

madalya kazanmayanları filtreleyip ülkeler bazında gruplama işlemi yaparak en çok madalya kazanan ülkeleri görelim:

In [ ]:
df_filtered = df[df.Medal != 'Medal Not Won']
medals_by_country = df_filtered.groupby('Team')['Medal'].count().sort_values(ascending=False)
medals_by_country

en çok madalya kazanan ülkelerden ilk 10 tanesini bar chart ile görselleştirelim:

In [ ]:
plt.figure(figsize=[18,8])
sns.barplot(x=medals_by_country[:10].index, y=medals_by_country[:10], palette="YlOrBr_r")
plt.title("Countries Won the Most Medals in the Olympics");

Ülkeler gönderdikleri sporcu sayılarına göre madalya kazanmakta ne kadar başarılı?:

In [ ]:
filtered_athlete = athlete_count[athlete_count > 1000]
country_success = (medals_by_country / filtered_athlete).sort_values(ascending=False).dropna()
country_success

şimdi bunu görselleştirelim:

In [ ]:
plt.figure(figsize=[15,10])
sns.barplot(x=country_success.values*100, y=country_success.index, palette='coolwarm_r')
plt.title("Countries Medal Won Percentage in the Olympics")
plt.xlabel("Percentage (%)");

yıllar içinde zamanla katılımcı sayılarımız nasıl değişmiş?

In [ ]:
athlete_by_year = df.groupby('Year')['Name'].count()
athlete_by_year


yıllar içindeki katılımcı sayısının değişimini line chart ile görselleştirelim:

In [ ]:
plt.figure(figsize=[18,8])
plt.xticks(np.linspace(1896,2016,13))
plt.grid()
sns.lineplot(x=athlete_by_year.index, y=athlete_by_year.values)
plt.title("Change in the Number of Athletes Over the Years")
plt.ylabel("Athlete Count");

spor dallarına göre kadın-erkek katılımcılarının arasındaki farkı görelim:

In [ ]:
plt.figure(figsize=(12, 5));
highest_sport = df['Sport'].value_counts().index
sns.countplot(data = df, x = 'Sport', hue = 'Sex', order = highest_sport, palette=sns.color_palette("seismic",2))
plt.xticks(rotation=90)
plt.title('Sports with Gender distribution')
plt.xlabel('Sport', fontweight='bold')
plt.ylabel('No. of Athletes', fontweight='bold');
plt.legend(['Male','Female'],loc=1, shadow=True);

erkek ve kadın sporcuların zamanla katılımına bakalım:

In [ ]:
year_wise_participants = df.groupby('Year')['Sex'].value_counts()
year_wise_participants.head(10)

kadın-erkek katılımcıların yıllar içindeki katılım düzeylerini bar chart ile görelim:

In [ ]:
plt.figure(figsize=(15,6))
sns.countplot(x = df['Year'], hue=df['Sex']);

hangi spor dalında hangi madalyadan kaç kişi ne kadar kazanmış?

In [ ]:
pd.DataFrame(df_filtered.groupby(['Sport','Medal']).Name.count().reset_index())

In [ ]:
medals_in_sports = df_filtered.groupby(['Sport','Medal']).Name.count().reset_index()
medals_in_sports = medals_in_sports.pivot_table(index='Sport', columns='Medal', values='Name')
medals_in_sports = medals_in_sports.replace(np.nan, 0)
medals_in_sports

Amerika nın yıllar içindeki madalya sayılarındaki değişimi görelim:

In [ ]:
usa = df[df['Team']=='United States']
usa

Amerika da sadece madalya kazananları görelim ve madalya kazanmayanları filtreleyelim:

In [ ]:
usa_medals_count = usa[usa['Medal'] !='Medal Not Won']
usa_medals_count

Amerika da hangi yıl kaç kişi madalya kazanmış?

In [ ]:
usa_medal = usa_medals_count.groupby('Year')['Medal'].count()
usa_medal

Amerika da hangi yıl kaç kişi madalya kazanmış line chart ile görselleştirelim

In [ ]:
plt.figure(figsize=[24,8])
plt.xticks(np.linspace(1896,2016,25))
plt.grid()
sns.lineplot(x=usa_medal.index, y=usa_medal.values);